# MNIST FCN with Virtual Branching

In [1]:
import tensorflow as tf
import numpy as np
import os
from scipy.special import softmax
import matplotlib.pyplot as plt

In [2]:
import vbranch as vb

In [3]:
save = False
model_id = 1

## Load Data

In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [5]:
input_dim = 784
num_classes = 10

In [6]:
X_train_flat = X_train.reshape([-1, input_dim])
X_test_flat = X_test.reshape([-1, input_dim])

y_train_one_hot = tf.keras.utils.to_categorical(y_train, num_classes)
y_test_one_hot = tf.keras.utils.to_categorical(y_test, num_classes)

## Train

### Build Model

In [7]:
BATCH_SIZE = 32
EPOCHS = 10
STEPS_PER_EPOCH = 100
NUM_BRANCHES = 1
model_path = './models/vb-mnist-B' + str(NUM_BRANCHES) + '_' + str(model_id)

In [8]:
model_path

'./models/vb-mnist-B1_1'

In [9]:
tf.reset_default_graph()

train_data = (X_train_flat.astype('float32'), y_train_one_hot)
test_data = (X_test_flat.astype('float32'), y_test_one_hot)

batch_size = tf.placeholder('int64', name='batch_size')

train_datasets = []
test_datasets = []
inputs = [None] * NUM_BRANCHES
labels_one_hot = [None] * NUM_BRANCHES
train_init_ops = []
test_init_ops = []

for i in range(NUM_BRANCHES):
    train_datasets.append(tf.data.Dataset.from_tensor_slices(train_data).\
        batch(batch_size).repeat().\
        shuffle(buffer_size=4*BATCH_SIZE))

    test_datasets.append(tf.data.Dataset.from_tensor_slices(test_data).\
        batch(batch_size))
    
    iterator = tf.data.Iterator.from_structure(train_datasets[i].output_types, 
                                           train_datasets[i].output_shapes)
    inputs[i], labels_one_hot[i] = iterator.get_next()    

    train_init_ops.append(iterator.make_initializer(train_datasets[i]))
    test_init_ops.append(iterator.make_initializer(test_datasets[i], 
                                                name='test_init_op_'+str(i+1)))

In [10]:
with tf.variable_scope('model_' + str(model_id), reuse=tf.AUTO_REUSE):
    model = vb.models.vbranch_fcn(inputs, ([128]*NUM_BRANCHES, 64), ([10]*NUM_BRANCHES, 5), 
                                  branches=NUM_BRANCHES)

In [11]:
model.summary()

i   Layer name          Output shapes                           Num param 
--------------------------------------------------------------------------
    Input               [None,784]                                        
--------------------------------------------------------------------------
0   fc1                 [None,64] [None,64]                     100480    
--------------------------------------------------------------------------
1   bn1                 [None,64] [None,64]                     256       
--------------------------------------------------------------------------
2   relu1               [None,64] [None,64]                     0         
--------------------------------------------------------------------------
3   fc2                 [None,5] [None,5]                       1300      
--------------------------------------------------------------------------
4   bn2                 [None,5] [None,5]                       20        
-------------------------

In [12]:
model.output

[<tf.Tensor 'model_1/output_1:0' shape=(?, 10) dtype=float32>]

In [13]:
tf.global_variables()

[<tf.Variable 'model_1/fc1_shared_to_shared_w:0' shape=(784, 64) dtype=float32_ref>,
 <tf.Variable 'model_1/fc1_shared_to_shared_b:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'model_1/fc1_vb1_shared_to_unique_w:0' shape=(784, 64) dtype=float32_ref>,
 <tf.Variable 'model_1/fc1_vb1_shared_to_unique_b:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'model_1/bn1_shared_to_shared_scale:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'model_1/bn1_shared_to_shared_beta:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'model_1/bn1_vb1_unique_to_unique_scale:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'model_1/bn1_vb1_unique_to_unique_beta:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'model_1/fc2_shared_to_shared_w:0' shape=(64, 5) dtype=float32_ref>,
 <tf.Variable 'model_1/fc2_shared_to_shared_b:0' shape=(5,) dtype=float32_ref>,
 <tf.Variable 'model_1/fc2_vb1_unique_to_shared_w:0' shape=(64, 5) dtype=float32_ref>,
 <tf.Variable 'model_1/fc2_vb1_unique_to_shared_b:0' shape=(5,) 

### Declare Training Ops

In [14]:
def get_shared_unshared_vars():
    """
    Get shared variables (in order to later average gradients)
    and unshared variables (unique to each branch)"""
    
    shared_vars = []
    unshared_vars = [[] for i in range(NUM_BRANCHES)]
    
    for var in tf.global_variables():
        if 'shared_to_shared' in var.name:
            shared_vars.append(var)
        else:
            for i in range(NUM_BRANCHES):
                if 'vb'+str(i+1) in var.name:
                    unshared_vars[i].append(var)
            
    return shared_vars, unshared_vars

In [15]:
shared_vars, unshared_vars = get_shared_unshared_vars()

In [16]:
unshared_vars

[[], [], [], []]

In [17]:
# Multi output loss
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

losses = []
# Store gradients from shared variables over each branch
shared_grads = []
unshared_train_ops = []

for i in range(NUM_BRANCHES):
    loss = vb.losses.softmax_cross_entropy_with_logits(labels=labels_one_hot[i], 
                                                       logits=model.output[i], 
                                                       name='loss_'+str(i+1))
    losses.append(loss)
    
    # Compute gradients of shared vars for each branch (but don't apply)
    if len(shared_vars) > 0:
        shared_grads.append(optimizer.compute_gradients(loss, var_list=shared_vars))
        
    # Apply gradients for unshared vars for each branch
    if len(unshared_vars[i]) > 0:
        unshared_train_ops.append(optimizer.minimize(loss, var_list=unshared_vars[i]))

In [18]:
# Take average of the gradients over each branch
mean_shared_grads = []

for v, var in enumerate(shared_vars):
    grad = tf.reduce_mean([shared_grads[i][v][0] for i in range(NUM_BRANCHES)], [0])
    mean_shared_grads.append((grad, var))
    
if len(shared_vars) > 0:
    shared_train_op = optimizer.apply_gradients(mean_shared_grads)
else:
    shared_train_op = []

In [19]:
mean_shared_grads

[(<tf.Tensor 'Mean:0' shape=(784, 128) dtype=float32>,
  <tf.Variable 'model_1/fc1_shared_to_shared_w:0' shape=(784, 128) dtype=float32_ref>),
 (<tf.Tensor 'Mean_1:0' shape=(128,) dtype=float32>,
  <tf.Variable 'model_1/fc1_shared_to_shared_b:0' shape=(128,) dtype=float32_ref>),
 (<tf.Tensor 'Mean_2:0' shape=(128,) dtype=float32>,
  <tf.Variable 'model_1/bn1_shared_to_shared_scale:0' shape=(128,) dtype=float32_ref>),
 (<tf.Tensor 'Mean_3:0' shape=(128,) dtype=float32>,
  <tf.Variable 'model_1/bn1_shared_to_shared_beta:0' shape=(128,) dtype=float32_ref>),
 (<tf.Tensor 'Mean_4:0' shape=(128, 10) dtype=float32>,
  <tf.Variable 'model_1/fc2_shared_to_shared_w:0' shape=(128, 10) dtype=float32_ref>),
 (<tf.Tensor 'Mean_5:0' shape=(10,) dtype=float32>,
  <tf.Variable 'model_1/fc2_shared_to_shared_b:0' shape=(10,) dtype=float32_ref>),
 (<tf.Tensor 'Mean_6:0' shape=(10,) dtype=float32>,
  <tf.Variable 'model_1/bn2_shared_to_shared_scale:0' shape=(10,) dtype=float32_ref>),
 (<tf.Tensor 'Mean_7:0

In [20]:
train_ops = [unshared_train_ops, shared_train_op]

In [21]:
# Train accuracies
train_acc_ops = []
for i in range(NUM_BRANCHES):
    pred_max = tf.one_hot(tf.argmax(tf.nn.softmax(model.output[i]), axis=-1), 
                          num_classes)
    train_acc_op = tf.reduce_mean(tf.reduce_sum(labels_one_hot[i]*pred_max, [1]), 
                                  name='train_acc_'+str(i+1))
    train_acc_ops.append(train_acc_op)

# Test accuracy
pred = tf.nn.softmax(tf.reduce_mean(model.output, [0]))
pred_max = tf.one_hot(tf.argmax(pred, axis=-1), num_classes)
test_acc_op = tf.reduce_mean(tf.reduce_sum(labels_one_hot[0]*pred_max, [1]), 
                             name='test_acc')

### Run Ops

In [22]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for e in range(EPOCHS):
        print("Epoch {}/{}".format(e + 1, EPOCHS))
        progbar = tf.keras.utils.Progbar(STEPS_PER_EPOCH)
        
        sess.run(train_init_ops, feed_dict={batch_size: BATCH_SIZE})

        for i in range(STEPS_PER_EPOCH):
            _, train_losses, train_accs = sess.run([train_ops, losses, 
                                                 train_acc_ops])
            
            prog_vals = [('loss_'+str(b+1),train_losses[b]) for b in range(NUM_BRANCHES)]
            prog_vals += [('acc_'+str(b+1),train_accs[b]) for b in range(NUM_BRANCHES)]
            
            if i == STEPS_PER_EPOCH - 1:
                sess.run(test_init_ops, feed_dict={batch_size: len(X_test_flat)})
                val_losses, val_acc, indiv_accs = sess.run([losses, test_acc_op, 
                                                            train_acc_ops])
                
                prog_vals += [("val_loss", np.mean(val_losses)), ("val_acc", val_acc)] + \
                    [('ind_acc_'+str(b+1), indiv_accs[b]) for b in range(NUM_BRANCHES)]
            
            progbar.update(i+1, values=prog_vals)
    
    if save:
        saver = tf.train.Saver()
        path = os.path.join(model_path, 'ckpt')
        saver.save(sess, path)

Epoch 1/10
100/100 [==============================] - 34s 337ms/step - loss_1: 0.9762 - loss_2: 0.9836 - loss_3: 0.9747 - loss_4: 0.9771 - acc_1: 0.8197 - acc_2: 0.8150 - acc_3: 0.8153 - acc_4: 0.8137 - val_loss: 0.6809 - val_acc: 0.9079 - ind_acc_1: 0.9079 - ind_acc_2: 0.9079 - ind_acc_3: 0.9079 - ind_acc_4: 0.9079
Epoch 2/10
100/100 [==============================] - 1s 13ms/step - loss_1: 0.6279 - loss_2: 0.6249 - loss_3: 0.6382 - loss_4: 0.6088 - acc_1: 0.9256 - acc_2: 0.9237 - acc_3: 0.9216 - acc_4: 0.9322 - val_loss: 0.5488 - val_acc: 0.9267 - ind_acc_1: 0.9267 - ind_acc_2: 0.9267 - ind_acc_3: 0.9267 - ind_acc_4: 0.9267
Epoch 3/10
100/100 [==============================] - 1s 11ms/step - loss_1: 0.4945 - loss_2: 0.4743 - loss_3: 0.4812 - loss_4: 0.4902 - acc_1: 0.9541 - acc_2: 0.9594 - acc_3: 0.9541 - acc_4: 0.9559 - val_loss: 0.4651 - val_acc: 0.9353 - ind_acc_1: 0.9353 - ind_acc_2: 0.9353 - ind_acc_3: 0.9353 - ind_acc_4: 0.9353
Epoch 4/10
100/100 [==============================

## Load Model

In [8]:
test_init_ops = ['test_init_op_'+str(i+1) for i in range(NUM_BRANCHES)]
losses = ['loss_'+str(i+1)+':0' for i in range(NUM_BRANCHES)]
train_acc_ops = ['train_acc_'+str(i+1)+':0' for i in range(NUM_BRANCHES)]

In [9]:
with tf.Session() as sess:
    model_path = './models/vb-mnist-B' + str(NUM_BRANCHES) + '_' + str(model_id)
    meta_path = os.path.join(model_path, 'ckpt.meta')
    ckpt = tf.train.get_checkpoint_state(model_path)

    imported_graph = tf.train.import_meta_graph(meta_path)
    imported_graph.restore(sess, ckpt.model_checkpoint_path)

    sess.run(test_init_ops, feed_dict={'batch_size:0': len(X_test_flat)})

    val_losses, val_acc, indiv_accs = sess.run([losses, 'test_acc:0', train_acc_ops])

INFO:tensorflow:Restoring parameters from ./models/vb-mnist-B4_1/ckpt


In [10]:
print('Loss:', np.mean(val_losses))
print('Acc:', val_acc)
print('Indiv accs:', indiv_accs)

Loss: 0.2564507
Acc: 0.9522
Indiv accs: [0.9434, 0.947, 0.9468, 0.945]
